In [14]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

model.invoke("Hello, world!")


AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 39, 'total_tokens': 64, 'completion_time': 0.090909091, 'prompt_time': 0.007609431, 'queue_time': 0.0002301579999999994, 'total_time': 0.098518522}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'stop', 'logprobs': None}, id='run-6703a612-100f-4639-b812-b98159a1c055-0', usage_metadata={'input_tokens': 39, 'output_tokens': 25, 'total_tokens': 64})

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import display, Markdown

messages = [
    SystemMessage("Create a masters dissertation draft according to the requirement from the human"),
    HumanMessage("jet noise interaction with the wing. experimental, semi-empirical, analytical and simulation approaches.")
]

# for token in model.stream(messages):
#     print(token.content, end="")
output = model.invoke(messages)
display(Markdown(output.content))


**Title:** Investigation of Jet Noise Interaction with the Wing: Experimental, Semi-Empirical, Analytical, and Simulation Approaches

**Abstract:**
The interaction between jet noise and the wing is a complex phenomenon that affects the overall noise levels and aerodynamic performance of an aircraft. This dissertation aims to investigate the jet noise interaction with the wing using experimental, semi-empirical, analytical, and simulation approaches. The study focuses on the noise reduction mechanisms and the effects of wing geometry, jet velocity, and angle of attack on the noise levels. The results of this research will contribute to the development of more efficient and quieter aircraft designs.

**Introduction:**
Jet noise is a significant contributor to the overall noise levels of an aircraft, and its interaction with the wing can lead to increased noise levels and reduced aerodynamic performance. The wing can act as a reflector, amplifier, or absorber of the jet noise, depending on its geometry and the flow conditions. Understanding the mechanisms of jet noise interaction with the wing is crucial for the development of more efficient and quieter aircraft designs.

**Literature Review:**
Previous studies have investigated the jet noise interaction with the wing using various approaches, including experimental, semi-empirical, analytical, and simulation methods. Experimental studies have focused on measuring the noise levels and flow fields around the wing-jet configuration. Semi-empirical models have been developed to predict the noise levels based on empirical correlations. Analytical models have been used to study the acoustic scattering and diffraction of the jet noise by the wing. Simulation approaches, such as computational fluid dynamics (CFD) and computational aeroacoustics (CAA), have been employed to simulate the flow and noise fields around the wing-jet configuration.

**Methodology:**
This study will employ a combination of experimental, semi-empirical, analytical, and simulation approaches to investigate the jet noise interaction with the wing. The experimental approach will involve measuring the noise levels and flow fields around a wing-jet configuration using a wind tunnel and acoustic measurement techniques. The semi-empirical approach will involve developing a model to predict the noise levels based on empirical correlations. The analytical approach will involve using acoustic scattering and diffraction theories to study the interaction between the jet noise and the wing. The simulation approach will involve using CFD and CAA to simulate the flow and noise fields around the wing-jet configuration.

**Experimental Approach:**
The experimental approach will involve measuring the noise levels and flow fields around a wing-jet configuration using a wind tunnel and acoustic measurement techniques. The wing-jet configuration will be mounted in a wind tunnel, and the noise levels will be measured using microphones and acoustic sensors. The flow fields will be measured using particle image velocimetry (PIV) and hot-wire anemometry. The experiments will be conducted at various jet velocities and angles of attack to study the effects of these parameters on the noise levels and flow fields.

**Semi-Empirical Approach:**
The semi-empirical approach will involve developing a model to predict the noise levels based on empirical correlations. The model will be developed using a combination of experimental data and empirical correlations. The model will be validated using experimental data and will be used to predict the noise levels for various wing-jet configurations.

**Analytical Approach:**
The analytical approach will involve using acoustic scattering and diffraction theories to study the interaction between the jet noise and the wing. The acoustic scattering and diffraction theories will be used to predict the noise levels and to study the effects of wing geometry and jet velocity on the noise levels.

**Simulation Approach:**
The simulation approach will involve using CFD and CAA to simulate the flow and noise fields around the wing-jet configuration. The simulations will be conducted using a commercial CFD software package, and the results will be validated using experimental data. The simulations will be used to study the effects of wing geometry, jet velocity, and angle of attack on the noise levels and flow fields.

**Results:**
The results of this study will be presented in terms of the noise levels and flow fields around the wing-jet configuration. The results will be compared with experimental data and will be used to validate the semi-empirical, analytical, and simulation models. The results will also be used to study the effects of wing geometry, jet velocity, and angle of attack on the noise levels and flow fields.

**Conclusion:**
This dissertation will contribute to the understanding of the jet noise interaction with the wing using experimental, semi-empirical, analytical, and simulation approaches. The results of this study will be used to develop more efficient and quieter aircraft designs. The study will also provide a comprehensive understanding of the mechanisms of jet noise interaction with the wing, which will be useful for the development of noise reduction technologies.

**Recommendations:**
Based on the results of this study, the following recommendations are made:

1. The wing geometry and jet velocity should be optimized to minimize the noise levels.
2. The angle of attack should be optimized to minimize the noise levels.
3. The semi-empirical, analytical, and simulation models should be used to predict the noise levels for various wing-jet configurations.
4. The results of this study should be used to develop more efficient and quieter aircraft designs.

**Future Work:**
Future work should focus on the following areas:

1. Developing more advanced semi-empirical, analytical, and simulation models to predict the noise levels and flow fields around the wing-jet configuration.
2. Investigating the effects of other parameters, such as wing camber and jet temperature, on the noise levels and flow fields.
3. Developing noise reduction technologies based on the understanding of the mechanisms of jet noise interaction with the wing.

**References:**

* [1] J. M. Seiner, "Aerodynamic noise from fluid flows," Journal of the Acoustical Society of America, vol. 74, no. 3, pp. 831-838, 1983.
* [2] A. S. Lyrintzis, "Theoretical modeling of jet noise," Journal of Fluid Mechanics, vol. 393, pp. 257-282, 1999.
* [3] J. C. Hardin, "Noise reduction by wing-jet interaction," Journal of Aircraft, vol. 24, no. 10, pp. 731-738, 1987.
* [4] Y. Liu, "Numerical simulation of jet noise and its interaction with the wing," Journal of Computational Physics, vol. 231, no. 10, pp. 4321-4334, 2012.

Note: This is a draft, and you should expand on each section and include more details, equations, and references as needed. Additionally, you should proofread and edit the text to ensure that it is clear, concise, and free of errors.

## Você pode criar templates de prompt, com placeholders para substituir automaticamente

In [3]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following sentence from English to {language}"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

In [4]:
prompt = prompt_template.invoke({"language": "brazilian portuguese", "text": "hey guys! how are you all doing?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following sentence from English to brazilian portuguese', additional_kwargs={}, response_metadata={}), HumanMessage(content='hey guys! how are you all doing?', additional_kwargs={}, response_metadata={})])

In [5]:
prompt.to_messages()

[SystemMessage(content='Translate the following sentence from English to brazilian portuguese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hey guys! how are you all doing?', additional_kwargs={}, response_metadata={})]

In [6]:
response = model.invoke(prompt)
print(response.content)

Ei pessoal! Como vocês estão?


# Output estruturado, permitindo o modelo escolher a estrutura mais adequada ao prompt

In [7]:
from typing import Optional, Union

from pydantic import BaseModel, Field
from enum import Enum

# Pydantic
class Joke(BaseModel):
    """Joke to tell the user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: int = Field(
        default=-1, description="How funny the joke is, from 1 to 10"
    )

class VoiceOptions(Enum):
    A = "creepy"
    B = "gentle"
    C = "misterious"
    D = "sad"

class Story(BaseModel):
    """Story to be told to the user."""

    soundtrack: str = Field(description="A specific song to be played while the story is read, to give the emotion the story evokes.")
    voice_option: VoiceOptions = Field(description="The tone that the person who reads the story must have to give the emotion the story evokes.")
    story: str = Field(description="The story to be read to the user")

class ConversationalResponse(BaseModel):
    """Respond in a conversational manner and be kind."""

    response: str = Field(description="A conversational response to the user's query.")

class AnswerOptions(BaseModel):
    answer: Union[Joke, Story, ConversationalResponse]

structured_llm = model.with_structured_output(AnswerOptions)

In [8]:
answer = structured_llm.invoke("Tell me a joke about cats")
answer

AnswerOptions(answer=Joke(setup='Why did the cat join a band?', punchline='Because it wanted to be the purr-cussionist!', rating=8))

In [9]:
answer = structured_llm.invoke("Tell me story about the sad life of a lonely ant")
answer

AnswerOptions(answer=Story(soundtrack='Mad World by Gary Jules', voice_option=<VoiceOptions.D: 'sad'>, story='Once upon a time, in a bustling colony of ants, there was one ant named Andy. Andy lived a life of solitude, often finding himself lost in the crowd of his own kin. While his fellow ants worked tirelessly to gather food and build their home, Andy struggled to find his place. He felt like an outcast, a small and insignificant speck in the grand scheme of things. As the days went by, Andy became increasingly withdrawn, spending most of his time alone, wandering the desolate outskirts of the colony. He longed for connection, for someone to share his thoughts and feelings with, but it seemed like no one was interested. The other ants were too busy with their own lives to notice Andy’s struggles. As the seasons changed, Andy’s loneliness only deepened. He felt like he was drowning in a sea of faces, unable to find a lifeline to cling to. His heart ached with a sadness that seemed to

In [10]:
answer.answer.voice_option.value

'sad'